# Bibliotecas Utilizadas

In [1]:
import spacy
#!python -m spacy download pt_core_news_sm
import pandas as pd
import string 
import numpy as np


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 53.6 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [2]:
nlp=spacy.load('pt_core_news_sm',disable=['paser','ner','tagger','textcat'])

# Modelos Importados

In [5]:
from gensim.models import KeyedVectors

In [6]:
w2v_modelo_cbow=KeyedVectors.load_word2vec_format("/kaggle/input/modelo-nlp-alura/modelo_cbow.txt")
w2v_modelo_skip=KeyedVectors.load_word2vec_format("/kaggle/input/modelo-nlp-alura/modelo_skip_gram.txt")

In [14]:
teste=pd.read_csv('/kaggle/input/alura-word2vec/teste.csv')
treino=pd.read_csv('/kaggle/input/alura-word2vec/treino.csv')

In [18]:
teste

,title,text,date,category,subcategory,link
0,Grandes irmãos,"RIO DE JANEIRO - O Brasil, cada vez menos famí...",2017-03-06,colunas,ruycastro,http://www1.folha.uol.com.br/colunas/ruycastro...
1,Haddad congela orçamento e suspende emendas de...,"O prefeito de São Paulo, Fernando Haddad (PT),...",2016-08-10,colunas,monicabergamo,http://www1.folha.uol.com.br/colunas/monicaber...
2,Proposta de reforma da Fifa tem a divulgação d...,"A Fifa divulgou, nesta quinta (10), um relatór...",2015-10-09,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/09/1...
3,"Mercado incipiente, internet das coisas conect...","Bueiros, coleiras, aparelhos hospitalares, ele...",2016-11-09,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/09/1...
4,"Mortes: Psicanalista, estudou o autismo em cri...",Toda vez que o grupo de amigos de Silvana Rabe...,2017-02-07,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2017/07...
...,...,...,...,...,...,...
20508,INSS diz que tempo médio de espera para períci...,O INSS (Instituto Nacional do Seguro Social) i...,2016-01-20,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/01/1...
20509,Erramos: O ditador que deu certo,Diferentemente do que deu a entender a coluna ...,2015-03-31,colunas,clovisrossi,http://www1.folha.uol.com.br/colunas/clovisros...
20510,Caixa Econômica elevou juros sob pressão de ba...,Executivos dos grandes bancos privados procura...,2016-08-04,mercado,NaN,http://www1.folha.uol.com.br/mercado/2016/04/1...
20511,Venda de carne 'in natura' para os Estados Uni...,"A exportação de carne ""in natura"" para os Esta...",2016-12-20,colunas,vaivem,http://www1.folha.uol.com.br/colunas/vaivem/20...


# Funções Utilizadas

In [3]:
def tokenizador(texto):
    doc=nlp(texto)
    tokens_validos=[]
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())
    
    
    return tokens_validos

In [7]:
texto=" Rio de janeiro ***** 4$$$ 2013 Maravilhoso"
tokens=tokenizador(texto)

In [9]:
def combinacao_de_vetores_por_soma(palavras, modelo):
    vetor_resultante = np.zeros(300) # Criando um vetor vazios de 300 posições
    for pn in palavras:
        #Soma o vetor de cada um tokens
        try:
            vetor_resultante += modelo.get_vector(pn)
        
        # Esses tratamentos são baseados no artigo.
        except KeyError:
                pass # ignora esse exceção
        
        
    return vetor_resultante

In [10]:
vetor_texto_cbow=combinacao_de_vetores_por_soma(tokens,w2v_modelo_cbow)
vetor_texto_sg=combinacao_de_vetores_por_soma(tokens,w2v_modelo_skip)

In [13]:
len(vetor_texto_cbow)

300

In [16]:
def matriz_vetores(textos,modelo):
    x=len(textos) # número de textos
    y= 300 # tamanho do vetor, fixo em 300
    matriz=np.zeros((x,y))
    
    # Percorre todos os textos
    for i in range(x):
        palavras= tokenizador(textos.iloc[i])
        matriz[i]=combinacao_de_vetores_por_soma(palavras,modelo)
    
    
    
    return matriz

In [20]:
titulos_cbow_treino= matriz_vetores(treino.title,w2v_modelo_cbow)
titulos_cbow_teste=matriz_vetores(teste.title,w2v_modelo_cbow)

In [24]:
titulos_sg_treino= matriz_vetores(treino.title,w2v_modelo_skip)
titulos_sg_teste=matriz_vetores(teste.title,w2v_modelo_skip)

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [28]:
def classificador(modelo,x_treino,y_treino,x_teste,y_teste):
    
    RL= LogisticRegression(max_iter=10000)
    RL.fit(x_treino,y_treino)
    categorias=RL.predict(x_teste)
    resultados=classification_report(y_teste,categorias)
    print(resultados)
    
    return RL



In [29]:
RL_cbow=classificador(w2v_modelo_cbow,
                     titulos_cbow_treino,
                     treino.category,
                     titulos_cbow_teste,
                     teste.category)

              precision    recall  f1-score   support

     colunas       0.76      0.67      0.72      6103
   cotidiano       0.58      0.74      0.65      1698
     esporte       0.87      0.79      0.83      4663
   ilustrada       0.08      0.77      0.15       131
     mercado       0.80      0.70      0.75      5867
       mundo       0.64      0.75      0.69      2051

    accuracy                           0.72     20513
   macro avg       0.62      0.74      0.63     20513
weighted avg       0.77      0.72      0.74     20513



In [30]:
RL_sg=classificador(w2v_modelo_skip,
                     titulos_sg_treino,
                     treino.category,
                     titulos_sg_teste,
                     teste.category)

              precision    recall  f1-score   support

     colunas       0.76      0.67      0.71      6103
   cotidiano       0.50      0.67      0.57      1698
     esporte       0.83      0.71      0.76      4663
   ilustrada       0.07      0.72      0.12       131
     mercado       0.77      0.65      0.70      5867
       mundo       0.54      0.67      0.60      2051

    accuracy                           0.67     20513
   macro avg       0.58      0.68      0.58     20513
weighted avg       0.73      0.67      0.69     20513



# Importando os modelos


In [31]:
import pickle

with open('/kaggle/working/rl_alura_cbow.pkl','wb') as f:
    pickle.dump(RL_cbow,f)

In [32]:
with open('/kaggle/working/rl_alura_sg.pkl','wb') as f:
    pickle.dump(RL_sg,f)